# Coastal Shoreline Condition

This is an indicator for the 2023 base blueprint.

Created by Blair Tirpak, modified from Shoreline Condition from Amy Keister, last run by Blair Tirpak on Feb 7 2022.

Updated Sep 18 2023 to update colors. 

In [1]:
import arcpy
from arcpy import env
from arcpy.sa import *

In [2]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
#SourceWorkspace= 
OutWorkspace = r"F:\Blueprint\2022\2022Indicators\Shoreline\2023\ShorelineCondition.gdb"

In [3]:
# define final indicator outputs
SEout = r"F:\Blueprint\2022\2022Indicators\Shoreline\2023\CoastalShorelineCondition.tif"

In [14]:
# define rasters used for cell size, extent, and snapping
SEraster= r"C:\Data\Indicators\BaseBlueprintExtent2022.tif"

In [14]:
# define inputs
ESI= r"C:\Data\Shoreline\NationalESI_Shoreline\GULF_ATLANTIC_ESI.gdb\GULF_ATLANTIC_ESI"
CBRS= r"C:\Data\Shoreline\CBRS-Boundaries\CBRS_Polygons_03122019.shp"

### Start Analysis

In [6]:
# Set the output workspace 
arcpy.env.workspace = OutWorkspace

In [7]:
# Print the current workspace to make sure I'm in the right spot
print(arcpy.env.workspace)

F:\Blueprint\2022\2022Indicators\Shoreline\2023\ShorelineCondition.gdb


## Bring in national ESI Shoreline data

In [9]:
# extract coastal shorelines, remove freshwater lines as well as lacustrine and riverine systems 
arcpy.FeatureClassToFeatureClass_conversion(ESI, OutWorkspace, "ESI", "LINE <> 'H' And ENVIR = 'E'")

<Result 'C:\\Data\\Shoreline\\ShorelineCondition.gdb\\ESI'>

Score line segements based on GENERALIZED_ESI_TYPE. This field contains 1-3 shoreline types, listed landward to seaward. If field contains only Armored, segment is considered fully armored. If Armored is anywhere along the shoreline, segment is considered partially armored. Otherwise segment is condidered natural. Confirmed by PI. 

In [49]:
# set code block for next step
codeblock = """
def Reclass(GENERALIZED_ESI_TYPE):
    if GENERALIZED_ESI_TYPE == "1: Armored":
        return 10
    if "Armored" in GENERALIZED_ESI_TYPE:
        return 20 
    else:
        return 30 
"""

In [50]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField("ESI", "Shore", "Reclass(!GENERALIZED_ESI_TYPE!)", "PYTHON3", codeblock, "SHORT")

<Result 'ESI'>

### <font color='red'>STOP CODE HERE. Manual edits were done to ESI to remove orphaned line segements just south of Baton Rouge. They were cut off after removing the mainstem of the Mississippi River.</font>

In [58]:
# convert to raster using "Shore" attribute. 
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SEraster, snapRaster=SEraster, cellSize=SEraster):
    arcpy.conversion.FeatureToRaster("ESI", "Shore", "ESIr", SEraster)

## Bring in CBRS data

In [52]:
# make a copy of the CBRS data so I can add a field
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SEraster):
    arcpy.management.CopyFeatures(CBRS, "CBRS", '', None, None, None)

In [53]:
# add a field to use for a dissolve and calculate it.
arcpy.management.CalculateField("CBRS", "dissolve", "2", "PYTHON3", '', "SHORT")

<Result 'CBRS'>

In [54]:
# convert to raster 
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SEraster, snapRaster=SEraster, cellSize=SEraster):
    arcpy.conversion.FeatureToRaster("CBRS", "dissolve", "CBRSr", SEraster)

In [ ]:
# copy Base Blueprint extent to use as background value to CBRS in order to add rasters
arcpy.CopyRaster_management(SEraster, "SEraster", '', None, "0", "NONE", "NONE", "8_BIT_UNSIGNED", "NONE", "NONE", "GRID", "NONE", "CURRENT_SLICE",
                            "NO_TRANSPOSE")

In [34]:
# mosaic CBRS to extent
arcpy.management.MosaicToNewRaster("CBRSr;SEraster", env.workspace, "CBRSRcls",
                                   None, "8_BIT_UNSIGNED", None, 1, "MAXIMUM", "FIRST")

<Result 'C:\\Data\\Shoreline\\ShorelineCondition.gdb\\CBRSRcls'>

## Combine rasters
here is what the classes will mean

- 32 as far as we know it isn't armored and it is in the CBRS
- 31 as far as we know it isn't armored and it is out of the CBRS
- 22 has the word armor somewhere in the GENERALIZED_ESI_TYPE field and is in the CBRS
- 21 has the word armor somewhere in the GENERALIZED_ESI_TYPE field and is out of the CBRS
- 12 has a value of 1 in the GENERAL_SYMBOL field and is in the CBRS
- 11 has a value of 1 in the GENERAL_SYMBOL field and is out of the CBRS

In [17]:
# add above rasters together
out_raster = arcpy.sa.Plus("ESIr", "CBRSRcls"); out_raster.save("Plus")

In [18]:
# reclassify raster. This is a guess at the priority, need to confirm with staff.
out_raster = arcpy.sa.Reclassify("Plus", "Value", "11 0;12 0;21 1;22 2;31 3;32 4", "DATA"); out_raster.save("indicator")

### Finalize indiator

do final steps for all indicators to add description fields, clip and export to SE extent, clip and export to SA extent

In [8]:
# set code block for next step
codeblock = """
def Reclass(value):
    if value == 4:
        return '4 = Natural and harder to develop'
    elif value == 3:
        return '3 = Natural' 
    elif value == 2:
        return '2 = Partially armored and harder to develop' 
    elif value == 1:
        return '1 = Partially armored' 
    elif value == 0:
        return '0 = Armored'  
"""

In [9]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField("indicator", "descript", "Reclass(!value!)", "PYTHON3", codeblock, "TEXT")

<Result 'F:\\Blueprint\\2022\\2022Indicators\\Shoreline\\2023\\ShorelineCondition.gdb\\indicator'>

In [10]:
# set code block for next step
codeblock = """
def Reclass1(Value):
	if Value == 4:
		return 0
	if Value == 3:
		return 50
	if Value == 2:
		return 115
	if Value == 1:
		return 172
	else:
		return 222
		
def Reclass2(Value):
	if Value == 4:
		return 92
	if Value == 3:
		return 125
	if Value == 2:
		return 162
	if Value == 1:
		return 179
	else:
		return 186
		
def Reclass3(Value):
	if Value == 4:
		return 77
	if Value == 3:
		return 23
	if Value == 2:
		return 44
	if Value == 1:
		return 101
	else:
		return 165
"""

In [11]:
# calculate Red field
arcpy.management.CalculateField("indicator", "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField("indicator", "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField("indicator", "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'F:\\Blueprint\\2022\\2022Indicators\\Shoreline\\2023\\ShorelineCondition.gdb\\indicator'>

In [15]:
# clip to SE 2022 Blueprint extent
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SEraster, snapRaster=SEraster, cellSize=SEraster):
    out_raster = arcpy.sa.ExtractByMask("indicator", SEraster); out_raster.save("SEMask")

In [16]:
# export as .tif with SE extent
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SEraster, snapRaster=SEraster, cellSize=SEraster):
    arcpy.management.CopyRaster("SEMask", SEout, '', None, "255", "NONE", "NONE", "8_BIT_UNSIGNED", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

# FIN